In [ ]:
# Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import glob
import rasterio
from rasterio import plot
from rasterio.plot import show
import os
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision.transforms as T
import matplotlib
from matplotlib.colors import ListedColormap
import pandas as pd
import geopandas as gpd
import gc
import torchsummary
from datetime import datetime
from torchvision.utils import save_image
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score

In [ ]:
#torch seed and device info
torch.random.manual_seed(0)
!nvidia-smi

In [ ]:
!nvidia-smi -L

In [ ]:
# Paths and data split
#images_dir = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/Input5bands"
images_dir = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/train4bands"
#masks_dir = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/Masks"
masks_dir = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/trainMasks"
#weights_path = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/weights"
weights_path = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/weightsTrainDS"
predictions_path = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/predictions"
valImgs = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/Input4bands"
valMasks = "/home/lcervantes/Dataset/Datasets/AI4boundariesDS/Masks"